# Expoloitation de donnnées

In [8]:
import pandas as pd

## DATA

### Capacité d'accueil

https://www.capareseau.fr/#

In [9]:
path_capa_accueil = '/Users/matthiasmolenat/repos/congestion/data/CapacitesDAccueil.csv'
df_capa_accueil = pd.read_csv(path_capa_accueil,delimiter=';')

In [10]:
df_capa_accueil.head()

,Code,Nom,S3REnR,Taux d'affection des capacite?s re?serve?es,X,Y,Puissance des projets en service du S3REnR en cours,Puissance des projets en developpement du S3REnR en cours,Capacite d'accueil reservee au titre du S3REnR qui reste e affecter,Puissance EnR deje raccordee,...,Tension aval.1,Tension amont.1,"Travaux GRD indiques dans le schema ou dans son etat initial, permettant d'augmenter la capacite reservee disponible.1",Puissance en file d'attente.1,Puissance en file d'attente hors S3REnR majoree de la capacite reservee du S3REnR.1,Capacite de transformation HTB/HTA restante disponible pour l'injection sur le reseau public de distribution.1,"Travaux indiques dans le schema ou dans son etat initial, permettant d'augmenter la capacite reservee disponible","Capacite d'accueil reservee au titre du S3REnR, disponible vue du reseau public de distribution en HTB",GRDHTB - Capacite d'accueil en HTB2,GRDHTB - Capacite d'accueil en HTB1
0,A.ADO,AIRE-SUR-ADOUR,NOUVELLE-AQUITAINE,73%,437096.44,6295185.5,3.5,4.1,1.1,11.7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A.BAI,AIX-LES-BAINS,AUVERGNE-RHONE-ALPES,37%,926603.25,6514719.5,0.3,0.1,9.5,2.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A.COM,AVESNES-LE-COMTE,HAUTS-DE-FRANCE,53%,665170.80,7019499.0,0.0,0.8,44.5,147.8,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A.DOU,ARNAC-SUR-DOURDOU,OCCITANIE,34%,696877.00,6294687.0,0.0,13.6,24.4,32.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A.DUC,ARNAY-LE-DUC,BOURGOGNE-FRANCHE-COMTe,57%,812437.56,6669952.0,1.5,4.6,28.5,31.5,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
df_capa_accueil.columns

Index(['Code', 'Nom', 'S3REnR', 'Taux d'affection des capacite?s re?serve?es',
       'X', 'Y', 'Puissance des projets en service du S3REnR en cours',
       'Puissance des projets en developpement du S3REnR en cours',
       'Capacite d'accueil reservee au titre du S3REnR qui reste e affecter',
       'Puissance EnR deje raccordee',
       'Puissance des projets EnR en developpement',
       'Capacite reservee aux EnR au titre du S3REnR',
       'Attention: la valeur de la capacite reservee a ete modifiee sur ce poste',
       'Quote-Part unitaire actualisee',
       'Attention : Le S3REnR de la region est sature',
       'dont la convention de raccordement est signee',
       'Capacite d'accueil reservee au titre du S3REnR, disponible vue du reseau public de transport ',
       'Travaux RTE indiques dans le schema ou dans son etat initial, permettant d'augmenter la capacite reservee disponible',
       'RTE - Capacite d'accueil en HTB2', 'RTE - Capacite d'accueil en HTB1',
       'Ca

### Contraintes réseau 

Par exemple pour Rhone Alpes :

https://www.contraintes-reseau-s3renr-rte.com/region/auvergne-rhône-alpes

In [15]:
path_contraintes_ARA = '/Users/matthiasmolenat/repos/congestion/data/contraintes_ARA.csv'
df_contraintes_ARA = pd.read_csv(path_contraintes_ARA, delimiter=';')

In [14]:
df_contraintes_ARA.head()

,,,,,Donnees de la region
Puissance ENR installee,Puissance Totale a compenser,Energie non-evacuee moyenne Printemps,Energie non-evacuee moyenne Ete,Energie non-evacuee moyenne Automne,Energie non-evacuee moyenne Hiver
13251MW,83.00MW,23.00MWh,1049.00MWh,14.00MWh,0.00MWh
Donnees de zones de la region,NaN,NaN,NaN,NaN,NaN
Zone,Puissance totale a compenser,Nombre de contraintes,Energie non-evacuee moyenne,NaN,NaN
Ain,6.00,3,526MWh/an,NaN,NaN
